In [7]:
!apt install build-essential libunbound-dev libidn2-dev libssl-dev cmake

!git clone https://github.com/getdnsapi/getdns.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
libssl-dev is already the newest version (3.0.2-0ubuntu1.15).
The following additional packages will be installed:
  libunbound8
The following NEW packages will be installed:
  libidn2-dev libunbound-dev libunbound8
0 upgraded, 3 newly installed, 0 to remove and 51 not upgraded.
Need to get 996 kB of archives.
After this operation, 3,319 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libunbound8 amd64 1.13.1-1ubuntu5.5 [399 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libunbound-dev amd64 1.13.1-1ubuntu5.5 [510 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libidn2-dev amd64 2.3.2-2build1 [86.7 kB]
Fetched 996 kB in 0s (2,508 kB/s)
Selecting previously unselected package libu

In [8]:
!cd getdns && mkdir build

In [14]:
!cd getdns && git submodule update --init

Submodule 'src/test/jsmn' (https://github.com/getdnsapi/jsmn.git) registered for path 'src/jsmn'
Submodule 'src/ssl_dane' (https://github.com/getdnsapi/ssl_dane) registered for path 'src/ssl_dane'
Submodule 'src/yxml' (https://github.com/getdnsapi/yxml.git) registered for path 'src/yxml'
Submodule 'stubby' (https://github.com/getdnsapi/stubby.git) registered for path 'stubby'
Cloning into '/content/getdns/src/jsmn'...
Cloning into '/content/getdns/src/ssl_dane'...
Cloning into '/content/getdns/src/yxml'...
Cloning into '/content/getdns/stubby'...
Submodule path 'src/jsmn': checked out '686a240cc8186a9a799ebafb0b32e67991b5abfc'
Submodule path 'src/ssl_dane': checked out '89ff45974cafa7e03f0503ed013fe3e680698c20'
Submodule path 'src/yxml': checked out '57497cdcacb6e5538e6bf6dad0b274896d9b3962'
Submodule path 'stubby': checked out 'b53867d1ec467822dbd9e1a0ad6598f84a9f5ed7'


In [15]:
!cd getdns/build && cmake .. && make && make install

-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found suitable version "3.0.2", minimum required is "1.0.2")  
-- Found OpenSSL: /usr/lib/x86_64-linux-gnu/libcrypto.so (found version "3.0.2")  
-- Could NOT find Libev (missing: LIBEV_LIBRARIES LIBEV_INCLUDE_DIR) 
CMake Warning at CMakeLists.txt:817 (message):
  Libev support library build requested, but libev not found.  Disabled.


-- Checking for module 'libuv'
--   No package 'libuv' found
-- Could NOT find Libuv (missing: LIBUV_LIBRARIES LIBUV_INCLUDE_DIR) 
CMake Warning at CMakeLists.txt:920 (message):
  Libuv support library build requested, but libuv not found.  Disabled.


-- Configuring done (0.5s)
-- Generating done (0.1s)
-- Build files have been written to: /content/getdns/build
[  1%] Building C object CMakeFiles/getdns_objects.dir/src/anchor.c.o
[  2%] Building C object CMakeFiles/getdns_objects.dir/src/const-info.c.o
[  4%] Building C object CMakeFiles/getdns_objects.dir/src/convert.c.o
[  5%] Building C object

In [12]:
!sudo apt-get install check

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
check is already the newest version (0.15.2-2).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [16]:
!apt update
!pip install getdns

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
51 packages can be upgraded. Run 'apt list --upgradable' to see them.
  Using cached getdns-v1.0.0b1.tar.gz (31 kB)
  Preparing metadata (setup.py) ... done
  Created w

In [17]:
import getdns
import sys
import csv

# Function to perform DNS resolution using getdns
def resolve_domain(domain, resolver_ip):
    try:
        # Create a context with the specified resolver
        context = getdns.Context()
        context.resolution_type = getdns.RESOLUTION_STUB
        context.upstream_recursive_servers = [
            {"address_data": resolver_ip, "address_type": "IPv4"}
        ]

        # Perform the DNS resolution
        results = context.address(name=domain)

        # Check if the query was successful
        if results.status == getdns.RESPSTATUS_GOOD:
            print(f"DNS resolution for {domain} succeeded using resolver {resolver_ip}.")
            for addr in results.just_address_answers:
                print(f"IP Address: {addr['address_data']}")
            return True
        else:
            print(f"DNS resolution for {domain} failed with status: {results.status} using resolver {resolver_ip}")
            return False
    except getdns.error as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        return False

# Function to read resolver IPs from a CSV file without headers
def read_csv(file_path):
    resolver_ips = []
    try:
        with open(file_path, mode='r') as file:
            csv_reader = csv.reader(file)
            for row in csv_reader:
                resolver_ips.append(row[0])
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.", file=sys.stderr)
    except Exception as e:
        print(f"An error occurred while reading the CSV file: {e}", file=sys.stderr)
    return resolver_ips

# Function to write successful resolver IPs to a CSV file
def write_csv(file_path, data):
    try:
        with open(file_path, mode='w', newline='') as file:
            csv_writer = csv.writer(file)
            for row in data:
                csv_writer.writerow([row])
    except Exception as e:
        print(f"An error occurred while writing to the CSV file: {e}", file=sys.stderr)

# Main function
def main():
    resolver_ips = ["8.8.8.8", "125.211.207.212", "185.175.49.49", "156.73.49.44", "132.246.16.136", "168.221.185.25"]

    file_path = 'results.csv'  # Path to your CSV file
    # resolver_ips = read_csv(file_path)

    domain = "90daykorean.com"  # Fixed domain to be resolved
    output_file_path = 'successful_resolvers.csv'
    # print(resolver_ips)

    successful_resolvers = []

    for resolver_ip in resolver_ips:
        if resolve_domain(domain, resolver_ip):
            successful_resolvers.append(resolver_ip)

    write_csv(output_file_path, successful_resolvers)

if __name__ == "__main__":
    main()

DNS resolution for 90daykorean.com succeeded using resolver 8.8.8.8.
IP Address: 2606:4700:20::681a:807
IP Address: 2606:4700:20::ac43:440a
IP Address: 2606:4700:20::681a:907
IP Address: 172.67.68.10
IP Address: 104.26.9.7
IP Address: 104.26.8.7
DNS resolution for 90daykorean.com failed with status: 902 using resolver 125.211.207.212
DNS resolution for 90daykorean.com succeeded using resolver 185.175.49.49.
IP Address: 2607:f740:100:108::1400
IP Address: 45.253.131.236
DNS resolution for 90daykorean.com failed with status: 902 using resolver 156.73.49.44
DNS resolution for 90daykorean.com failed with status: 902 using resolver 132.246.16.136
DNS resolution for 90daykorean.com failed with status: 902 using resolver 168.221.185.25


In [49]:
!sudo apt-get install geoip-bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
geoip-bin is already the newest version (1.6.12-8).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.
GeoIP Country Edition: GB, United Kingdom


In [61]:
!geoiplookup 125.211.207.212

GeoIP Country Edition: US, United States


In [67]:
!curl ipinfo.io/125.211.207.212?token=04c76ddfcb630d

{
  "ip": "125.211.207.212",
  "city": "Harbin",
  "region": "Heilongjiang",
  "country": "CN",
  "loc": "45.7500,126.6500",
  "postal": "150000",
  "timezone": "Asia/Shanghai",
  "asn": {
    "asn": "AS4837",
    "name": "CHINA UNICOM China169 Backbone",
    "domain": "chinaunicom.cn",
    "route": "125.211.0.0/16",
    "type": "isp"
  },
  "company": {
    "name": "China Unicom Heilongjiang Province Network",
    "domain": "chinaunicom.cn",
    "type": "isp"
  },
  "privacy": {
    "vpn": false,
    "proxy": false,
    "tor": false,
    "relay": false,
    "hosting": false,
    "service": ""
  },
  "abuse": {
    "address": "No.21,Financial Street, Beijing,100033, P.R.China",
    "country": "CN",
    "email": "zhaoyz3@chinaunicom.cn",
    "name": "ChinaUnicom Hostmaster",
    "network": "125.211.0.0/16",
    "phone": "+86-10-66259764"
  },
  "domains": {
    "total": 0,
    "domains": []
  }
}